In [1]:
!git clone https://github.com/openai/shap-e
%cd shap-e
!pip install -e .

Cloning into 'shap-e'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (260/260), done.
remote: Compressing objects: 100% (241/241), done.
remote: Total 336 (delta 41), reused 218 (delta 18), pack-reused 76
Receiving objects: 100% (336/336), 11.72 MiB | 12.19 MiB/s, done.
Resolving deltas: 100% (44/44), done.
/kaggle/working/shap-e
Obtaining file:///kaggle/working/shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-4h7bzy1r/clip_088db4b6793d49b5a7d5fe3e9e3cb437
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-4h7bzy1r/clip_088db4b6793d49b5a7d5fe3e9e3cb437
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
   ━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
from shap_e.util.image_util import load_image

In [3]:
class CFG:
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    BASE_NAME1 = 'transmitter'
    BASE_NAME2 = 'image300M'
    batch_size = 4
    guidance_scale = 3.0
    size = 64

In [4]:
xm = load_model(CFG.BASE_NAME1, device = CFG.DEVICE)
model = load_model(CFG.BASE_NAME2, device = CFG.DEVICE)
diffusion = diffusion_from_config(load_config('diffusion'))

  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|████████████████████████████████████████| 890M/890M [00:09<00:00, 101MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

In [5]:

# To get the best result, you should remove the background and show only the object of interest to the model.
image = load_image('/kaggle/input/muhimagesample/cube_stack.jpg')

latents = sample_latents(
    batch_size = CFG.batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale = CFG.guidance_scale,
    model_kwargs=dict(images=[image] * CFG.batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [6]:
render_mode = 'nerf' # you can change this to 'stf' for mesh rendering

cameras = create_pan_cameras(CFG.size, CFG.DEVICE)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode = render_mode)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAOYAAOUAAOQAAOMAAOIAAOAAAN8AANwAANsAANoAANkAANgAAN…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAOUAAOQAAOMAAOIAAOEAAOAAANsAANoAANkAANgABtgAAdgAAN…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAPXvAPTuAPPuAPLtAOzqAefmAOblAOXlAOXkAOTjAOPjAOLiAO…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAOUAAOQAAOMAAOIAAOEAAN8AANwAANgAANcAAdcAANYAA9YAAN…